<a href="https://colab.research.google.com/github/Barros313/Big-Data-1---Senac-TADS028/blob/main/BigData1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criação do Banco de Dados

Importação de bibliotecas externas e criação de função para fazer o banco de dados em SQL a partir de banco de dados em CSV

In [1]:
import requests
import sqlite3
import pandas as pd

def create_infracoes_database():
  url = 'http://dados.recife.pe.gov.br/api/3/action/datastore_search?resource_id=c269789d-da47-4dde-8ce7-42fba10fe8e2'

  response = requests.get(url)
  data = response.json()

  if 'result' in data and 'records' in data['result']:
    infracoes = data['result']['records']
  else:
    print("Erro: não foi possível obter dados da API")
    return

  conn = sqlite3.connect('infracoes.db')
  c = conn.cursor()

  c.execute('''
    CREATE TABLE IF NOT EXISTS infracoes (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      datainfracao DATETIME,
      horainfracao DATETIME,
      dataimplantacao DATETIME,
      agenteequipamento TEXT,
      infracao INT,
      descricaoinfracao TEXT,
      amparolegal TEXT,
      localcometimento TEXT
    )
  ''')

  for infracao in infracoes:
    c.execute('''
      INSERT INTO infracoes (
        datainfracao,
        horainfracao,
        dataimplantacao,
        agenteequipamento,
        infracao,
        descricaoinfracao,
        amparolegal,
        localcometimento)
      VALUES (?, ?, ?, ?, ?, ?, ?, ?);
    ''', (
        infracao.get("datainfracao", ""),
        infracao.get("horainfracao", ""),
        infracao.get("dataimplantacao", ""),
        infracao.get("agenteequipamento", ""),
        infracao.get("infracao", 0),
        infracao.get("descricaoinfracao", ""),
        infracao.get("amparolegal", ""),
        infracao.get("localcometimento", "")
        ))

  conn.commit()
  conn.close()

create_infracoes_database()



# Consulta geral

Consultando tabela única (infrações) do banco de dados

In [9]:
connection = sqlite3.connect('infracoes.db')

pd.read_sql("SELECT * FROM infracoes;", connection, index_col=["id"])

,datainfracao,horainfracao,dataimplantacao,agenteequipamento,infracao,descricaoinfracao,amparolegal,localcometimento
id,,,,,,,,
1,2023-01-01T00:00:00,00:01:00,2023-01-06T00:00:00,Código 8 - AUTOS NO TALÃO ELETRÔNICO,7234,Quando o veículo estiver em movimento deixar d...,"Art. 250, Inc. I, alínea a","RUA RIBEIRO DE BRITO, SOB O SEMAFORO N. 155"
2,2023-01-01T00:00:00,00:27:00,2023-01-06T00:00:00,Código 8 - AUTOS NO TALÃO ELETRÔNICO,5835,Desobedecer às ordens emanadas da autoridade c...,Art. 195,"RUA DOS NAVEGANTES, CRUZAMENTO COM A RUA PADRE..."
3,2023-01-01T00:00:00,00:27:00,2023-01-06T00:00:00,Código 8 - AUTOS NO TALÃO ELETRÔNICO,5215,Dirigir ameaçando os pedestres que estejam atr...,Art. 170,"RUA DOS NAVEGANTES, CRUZAMENTO COM A RUA PADRE..."
4,2023-01-01T00:00:00,00:52:00,2023-01-06T00:00:00,Código 8 - AUTOS NO TALÃO ELETRÔNICO,5550,Estacionar o veículo em locais e horários proi...,"Art. 181, Inc. XVIII","AVENIDA ENGENHEIRO DOMINGOS FERREIRA, EM FRENT..."
5,2023-01-01T00:00:00,00:52:00,2023-01-06T00:00:00,Código 8 - AUTOS NO TALÃO ELETRÔNICO,5550,Estacionar o veículo em locais e horários proi...,"Art. 181, Inc. XVIII","AVENIDA ENGENHEIRO DOMINGOS FERREIRA, EM FRENT..."
...,...,...,...,...,...,...,...,...
96,2023-01-01T00:00:00,06:48:43,2023-01-13T00:00:00,Código 3 - LOMBADA ELETRÔNICA,7455,Transitar em velocidade superior à máxima perm...,"Art. 218, Inc. I","AV. ENG. ABDIAS DE CARVALHO, SEMAFORO 272, Sen..."
97,2023-01-01T00:00:00,06:49:22,2023-01-13T00:00:00,Código 3 - LOMBADA ELETRÔNICA,7455,Transitar em velocidade superior à máxima perm...,"Art. 218, Inc. I","AV. ENG. ABDIAS DE CARVALHO, SEMAFORO 272, Sen..."
98,2023-01-01T00:00:00,06:51:30,2023-01-13T00:00:00,Código 3 - LOMBADA ELETRÔNICA,7455,Transitar em velocidade superior à máxima perm...,"Art. 218, Inc. I","AV. HERCULANO BANDEIRA, SEMAFORO 137., Faixa: 3"


# Função de query SQL

Criação de função que deixa a conexão com o banco de dados usado como padrão.

In [5]:
def sq(str, con=connection):
  return pd.read_sql('''{}'''.format(str), con)

# Características da Tabela

Usando comando 'PRAGMA table_info()' para fazer o papel do comando 'DESCRIBE' do MySQL para mostrar as características da tabela do banco de dados

In [8]:
sq("PRAGMA table_info([infracoes]);")

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,datainfracao,DATETIME,0,None,0
2,2,horainfracao,DATETIME,0,None,0
3,3,dataimplantacao,DATETIME,0,None,0
4,4,agenteequipamento,TEXT,0,None,0
5,5,infracao,INT,0,None,0
6,6,descricaoinfracao,TEXT,0,None,0
7,7,amparolegal,TEXT,0,None,0
8,8,localcometimento,TEXT,0,None,0
